In [20]:
import random

import torch

import src.models as models
from src.audio_augmentations import PitchShiftAugmentation, TimeStretchAugmentation
from src.audio_dataset import get_dataloader
from src.trainer import Trainer


In [48]:
# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wavecnn5 = getattr(models, 'WaveCNN5')
wavecnn5 = wavecnn5().to(device)

wavecnn7 = getattr(models, 'WaveCNN7')
wavecnn7 = wavecnn7().to(device)

wavecnn9 = getattr(models, 'WaveCNN9')
wavecnn9 = wavecnn9().to(device)

melcnn4 = getattr(models, 'FullyConvNet4')
melcnn4 = melcnn4().to(device)

melcnn5 = getattr(models, 'FullyConvNet5')
melcnn5 = melcnn5().to(device)


In [38]:
# Load Mel Spec 
# Initialize the Trainer
trainer5 = Trainer(wavecnn5, train_loader=None, valid_loader=None, learning_rate=0.001, device=device)
# Load model
trainer5.load_model('models/WaveCNN5_epoch10_20231127-0100.pth')

# Initialize the Trainer
trainer7 = Trainer(wavecnn7, train_loader=None, valid_loader=None, learning_rate=0.001, device=device)
# Load model
trainer7.load_model('models/waveform_cnn7.pth')
# 
# # Initialize the Trainer
# trainer9 = Trainer(wavecnn9, train_loader=None, valid_loader=None, learning_rate=0.001, device=device)
# # Load model
# trainer9.load_model('models/waveform_cnn9.pth')

# Initialize the Trainer
trainer_mel4 = Trainer(melcnn4, train_loader=None, valid_loader=None, learning_rate=0.001, device=device)
# Load model
trainer_mel4.load_model('models/spec_fcn4_aug_20231122-2022.pth')

In [39]:
import matplotlib.pyplot as plt

# Sample data
# Replace these with your actual data
epochs = range(1, 11)  # Example epoch range from 1 to 10

# Plotting
plt.figure(figsize=(12, 5))

# Plot for val_roc_auc
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(epochs, trainer5.history['val_roc_auc'], label='Wavefrom CNN 5 Layers ROC AUC')
plt.plot(epochs, trainer7.history['val_roc_auc'], label='Wavefrom CNN 7 Layers ROC AUC')
plt.plot(epochs, trainer_mel4.history['val_roc_auc'], label='MelSpectogram FCN 4 Layers ROC AUC')
plt.title('Validation ROC AUC over Epochs')
plt.xlabel('Epochs')
plt.ylabel('ROC AUC')
plt.legend()

# Plot for val_pr_auc
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(epochs, trainer5.history['val_pr_auc'], label='Wavefrom CNN 5 Layers PR AUC')
plt.plot(epochs, trainer7.history['val_pr_auc'], label='Wavefrom CNN 7 Layers PR AUC')
plt.plot(epochs, trainer_mel4.history['val_pr_auc'], label='MelSpectogram FCN 4 Layers ROC AUC')
plt.title('Validation PR AUC over Epochs')
plt.xlabel('Epochs')
plt.ylabel('PR AUC')
plt.legend()

plt.tight_layout()
plt.show()


### Evaluate

In [45]:
test_annotations = 'mtat_test_label.csv'
data_dir = 'data'
    
# Ealuate the model on the test set
test_loader = get_dataloader(
    annotations_file=test_annotations,
    data_dir=data_dir,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    sample_rate=16000,
    target_length=29.1,
)


In [41]:
avg_loss, roc_auc, pr_auc, predicted_labels, true_labels, filepaths = trainer_mel4.evaluate(test_loader)

LibsndfileError: Error opening 'data\\mtat/e/seth_carlin-mozart_in_the_age_of_enlightenment-14-sonata_in_bflat_major__kv_333__andante_cantabile_wolfgang_amadeus_mozart-30-59.mp3': System error.